In [1]:
import pandas as pd
import json
import io
from pandas.io.json import json_normalize
import pymongo 
from pymongo import MongoClient
import pprint
import numpy as np

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.dataton2019
collection = db.S6_Contrataciones
contrataciones = collection.find()

### RedFlag 1:

Se consideran las contrataciones donde la informacion de contacto de dos o mas porstores coincide, ya sea email o numero de telefono.  

In [3]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redflag_1 = []
for i in contrataciones:
    if(len(i['parties'])>3):
        x=[]
        for n in range(len(i['parties'][2:])):
            for j in range(len(i['parties'][2:])):
                if(j != n):
                    try:
                        y = i['parties'][2+n]['contactPoint']['email']
                        x.append(i['parties'][2+n]['contactPoint']['email'].split('; ')[:][0]==i['parties'][2+j]['contactPoint']['email'].split('; ')[:][0])
                    except:
                        break
        if(True in x):
            redflag_1.append(1)
        else:
            redflag_1.append(0)
    else: 
        redflag_1.append(0)

In [4]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redflag_uno = []
for i in contrataciones:
    if(len(i['parties'])>3):
        x=[]
        for n in range(len(i['parties'][2:])):
            for j in range(len(i['parties'][2:])):
                if(j != n):
                    try:
                        y = i['parties'][2+n]['contactPoint']['telephone']
                        x.append(i['parties'][2+n]['contactPoint']['telephone'].split('  ')[:][0]==i['parties'][2+j]['contactPoint']['telephone'].split('  ')[:][0])
                    except:
                        break
        if(True in x):
            redflag_uno.append(1)
        else:
            redflag_uno.append(0)
    else: 
        redflag_uno.append(0)

In [5]:
redFlag1 = []
for i in range(len(redflag_1)):
    if(redflag_1[i]==1 or redflag_uno[i]==1):
        redFlag1.append(1)
    else:
        redFlag1.append(0)

### RedFlag 2:

Se consideran las contrataciones donde la información de contacto del proveedor coincide con la informacion de la entidad contratante

In [6]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redflag_2 = []
for i in contrataciones:
    y = 0
    x = i['parties'][1]['contactPoint']['email'].split('; ')[:][0]
    for j in range(len(i['parties'])):
        if(len(i['parties'][j]['roles'])==2):
            try:
                y = i['parties'][j]['contactPoint']['email'].split('; ')[:][0] 
            except:
                break
    if(x==y):
        redflag_2.append(1)
    else:
        redflag_2.append(0)      

In [7]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redflag_dos = []
for i in contrataciones:
    y = 0
    x = i['parties'][1]['contactPoint']['telephone'].split('  ')[:][0]
    for j in range(len(i['parties'])):
        if(len(i['parties'][j]['roles'])==2):
            try:
                y = i['parties'][j]['contactPoint']['telephone'].split('  ')[:][0] 
            except:
                break
    if(x==y):
        redflag_dos.append(1)
    else:
        redflag_dos.append(0)

In [8]:
redFlag2 = []
for i in range(len(redflag_1)):
    if(redflag_2[i]==1 or redflag_dos[i]==1):
        redFlag2.append(1)
    else:
        redFlag2.append(0)

### RedFlag 3

Se muestran las contrataciones que superan el limite presupuestal

In [9]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redFlag3 = []
lim = 4000000
for i in contrataciones:
    try:
        x = i['tender']['procurementMethod']
        if(x == 'direct'):
            try:
                y = i['contracts'][0]['value']['amount']
                if(y > lim):
                    redFlag3.append(1)
                else:
                    redFlag3.append(0)
            except:
                redFlag3.append(0)
        else:
            redFlag3.append(0)
    except:
        redFlag3.append(0)

### RedFlag 4:

Se consideran las contrataciones donde existen cnoincidencias de apellido entre el servidor publico que contrata y el provedor.

In [10]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
redFlag4 = []
for i in contrataciones:
    y=0
    x = i['parties'][1]['contactPoint']['name'].lower().split(' ')[-2:]
    for j in range(len(i['parties'])):
        if(len(i['parties'][j]['roles'])==2):
            try:
                y = i['parties'][j]['contactPoint']['name'].lower().split(' ')[:][-2:]
            except:
                break
    if(x==y):
        redFlag4.append(1)
    else: 
        redFlag4.append(0)

In [11]:
collection = db.S6_Contrataciones
contrataciones = collection.find()
ocid = []
for i in contrataciones:
    ocid.append(i['ocid'])

### Data Frame con las contrataciones que caen en alguna bandera roja

In [12]:
df = pd.DataFrame({'ocid': ocid, 'red_flag_1': redFlag1, 'red_flag_2': redFlag2, 'red_flag_3': redFlag3, 'red_flag_4': redFlag4})

In [19]:
df2 = df[(df['red_flag_1']==1) | (df['red_flag_2']==1) | (df['red_flag_3']==1) | (df['red_flag_4']==1)]
df2.head()

,ocid,red_flag_1,red_flag_2,red_flag_3,red_flag_4
0,ocds-07smqs-1003803,0,0,1,0
4,ocds-07smqs-1025654,0,0,1,0
5,ocds-07smqs-1017118,0,0,1,0
42,ocds-07smqs-1117530,0,0,1,0
51,ocds-07smqs-1150878,0,0,1,0


In [14]:
df2.to_csv('grafo_redFlags_final_final.csv')